In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
from matplotlib import pyplot as plt

In [2]:
# set the number of diplayed columns in jupyter notebook to 100
pd.set_option('display.max_columns', 100)

In [3]:
df_train = pd.read_csv('data/training_set_VU_DM.csv')
df_train['is_test'] = False
df_test = pd.read_csv('data/test_set_VU_DM.csv')
df_test['is_test'] = True

In [4]:
# There is no correspondence between the srch_id's of the train and test sets, so we need to make them unique for further aggregation purposes
TEST_SRCH_ID_OFFSET = df_train.srch_id.max()
df_test['srch_id'] += TEST_SRCH_ID_OFFSET
assert len(set(df_test.srch_id.unique()).intersection(set(df_train.srch_id.unique()))) == 0

In [5]:
df_joined = pd.concat([df_train, df_test], axis=0, ignore_index=True)
df_joined.shape

(9917530, 55)

In [6]:

#del df_train, df_test
# call the garbage collector to free up memory
import gc
gc.collect()

0

In [7]:
df = df_joined.copy()
gc.collect()

0

# NAs

In [8]:
def print_missing(df, col):
    miss_cnt = df[col].isna().sum()
    total_cnt = len(df[col])
    perc_miss = 100 * miss_cnt / total_cnt
    print(f'{col} missing count {miss_cnt} out of {total_cnt} => {round(perc_miss)}% missing')

for col in [c for c in df.columns if df[c].isna().any() and not c.startswith('comp')]:
    print_missing(df, col)

visitor_hist_starrating missing count 9412233 out of 9917530 => 95% missing
visitor_hist_adr_usd missing count 9409918 out of 9917530 => 95% missing
prop_review_score missing count 14630 out of 9917530 => 0% missing
prop_location_score2 missing count 2178380 out of 9917530 => 22% missing
position missing count 4959183 out of 9917530 => 50% missing
srch_query_affinity_score missing count 9281966 out of 9917530 => 94% missing
orig_destination_distance missing count 3216461 out of 9917530 => 32% missing
click_bool missing count 4959183 out of 9917530 => 50% missing
gross_bookings_usd missing count 9779140 out of 9917530 => 99% missing
booking_bool missing count 4959183 out of 9917530 => 50% missing


In [9]:
# zero have special meaning, we don't want this value to interfere with various calculations, we will deal with NAs in derived values later
df['prop_starrating_w0'] = df['prop_starrating'].copy()
df.loc[df.prop_starrating_w0==0, 'prop_starrating_w0'] = pd.NA
df.loc[df.prop_review_score==0, 'prop_review_score'] = pd.NA

In [10]:
for col in [c for c in df.columns if df[c].isna().any() and not c.startswith('comp')]:
    print_missing(df, col)

# prop_starrating missing count 337794 out of 9917530 => 3% missing
# prop_review_score missing count 482116 out of 9917530 => 5% missing

visitor_hist_starrating missing count 9412233 out of 9917530 => 95% missing
visitor_hist_adr_usd missing count 9409918 out of 9917530 => 95% missing
prop_review_score missing count 482116 out of 9917530 => 5% missing
prop_location_score2 missing count 2178380 out of 9917530 => 22% missing
position missing count 4959183 out of 9917530 => 50% missing
srch_query_affinity_score missing count 9281966 out of 9917530 => 94% missing
orig_destination_distance missing count 3216461 out of 9917530 => 32% missing
click_bool missing count 4959183 out of 9917530 => 50% missing
gross_bookings_usd missing count 9779140 out of 9917530 => 99% missing
booking_bool missing count 4959183 out of 9917530 => 50% missing
prop_starrating_w0 missing count 337794 out of 9917530 => 3% missing


# Reducing number of columns

In [11]:
# Aggregate competitor information - perhaps not

# # add flag to indicate if any competitor has availability at a better rate
# for i in range(1, 9):
#     df[f'comp{i}_known'] = ~(df[f'comp{i}_rate'].isna() | df[f'comp{i}_inv'].isna())
#     df[f'comp{i}_better'] = df[f'comp{i}_known'] & (df[f'comp{i}_rate']==-1) & (df[f'comp{i}_inv']<=0)
#     df[f'comp{i}_worse'] = df[f'comp{i}_known'] & (df[f'comp{i}_rate']==1) & (df[f'comp{i}_inv']>=0)
#
# df['comp_known_cnt'] = sum([df[f'comp{i}_known'].astype(int) for i in range(1, 9)])
# df['comp_better_worse'] = \
#     (sum([df[f'comp{i}_better'].astype(int) for i in range(1, 9)])
#      -sum([df[f'comp{i}_worse'].astype(int) for i in range(1, 9)]))

In [12]:
#comp_rate_cols = [f'comp{i}_rate' for i in range(1, 9)]
#df['comp_rate_sum'] = df[comp_rate_cols].fillna(0).sum(axis=1)

In [13]:
# TODO: we could get rid of 'comp_better_cnt' > X since their effect on booking and click probability seems similar
# df['comp_better_cnt'] = np.miminum(df['comp_better_cnt'], 4)

# TODO: we could get rid of 'comp_known_cnt' > X
# df['comp_known_cnt'] = np.miminum(df['comp_known_cnt'], 4)

In [14]:
# TODO: add best discount
# for i in range(1, 9):
#     df[f'comp{i}_discount'] = \
#         (df[f'comp{i}_known']).astype(int) \
#         * (df[f'comp{i}_inv']<=0).astype(int) \
#         * (df[f'comp{i}_rate']!=0).astype(int) \
#         * (df[f'comp{i}_rate_percent_diff']<55).astype(int) \
#         * -1 * df[f'comp{i}_rate'] \
#         * df[f'comp{i}_rate_percent_diff']
#
# df['comp_best_discount'] = df[[f'comp{i}_discount' for i in range(1, 9)]].max(axis=1)
# df['comp_best_discount'].fillna(0, inplace=True)

In [15]:
#comps = [f'comp{i}_' for i in range(1, 9)]
#df.drop(columns=[c for c in df.columns if c[:6] in comps], inplace=True)

In [16]:
df.drop(columns=['gross_bookings_usd'], inplace=True)

In [17]:
# TODO: some aggregation might work?
df.drop(columns=['orig_destination_distance'], inplace=True)

In [18]:
df[df.prop_log_historical_price == 0]['prop_log_historical_price'] = pd.NA

/tmp/ipykernel_906532/456885427.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[df.prop_log_historical_price == 0]['prop_log_historical_price'] = pd.NA


In [19]:
df

,srch_id,date_time,site_id,visitor_location_country_id,visitor_hist_starrating,visitor_hist_adr_usd,prop_country_id,prop_id,prop_starrating,prop_review_score,prop_brand_bool,prop_location_score1,prop_location_score2,prop_log_historical_price,position,price_usd,promotion_flag,srch_destination_id,srch_length_of_stay,srch_booking_window,srch_adults_count,srch_children_count,srch_room_count,srch_saturday_night_bool,srch_query_affinity_score,random_bool,comp1_rate,comp1_inv,comp1_rate_percent_diff,comp2_rate,comp2_inv,comp2_rate_percent_diff,comp3_rate,comp3_inv,comp3_rate_percent_diff,comp4_rate,comp4_inv,comp4_rate_percent_diff,comp5_rate,comp5_inv,comp5_rate_percent_diff,comp6_rate,comp6_inv,comp6_rate_percent_diff,comp7_rate,comp7_inv,comp7_rate_percent_diff,comp8_rate,comp8_inv,comp8_rate_percent_diff,click_bool,booking_bool,is_test,prop_starrating_w0
0,1,2013-04-04 08:32:15,12,187,NaN,NaN,219,893,3,3.5,1,2.83,0.0438,4.95,27.0,104.77,0,23246,1,0,4,0,1,1,NaN,1,NaN,NaN,NaN,0.0,0.0,NaN,0.0,0.0,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN,0.0,0.0,False,3.0
1,1,2013-04-04 08:32:15,12,187,NaN,NaN,219,10404,4,4.0,1,2.20,0.0149,5.03,26.0,170.74,0,23246,1,0,4,0,1,1,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,0.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN,0.0,0.0,False,4.0
2,1,2013-04-04 08:32:15,12,187,NaN,NaN,219,21315,3,4.5,1,2.20,0.0245,4.92,21.0,179.80,0,23246,1,0,4,0,1,1,NaN,1,NaN,NaN,NaN,0.0,0.0,NaN,0.0,0.0,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN,0.0,0.0,False,3.0
3,1,2013-04-04 08:32:15,12,187,NaN,NaN,219,27348,2,4.0,1,2.83,0.0125,4.39,34.0,602.77,0,23246,1,0,4,0,1,1,NaN,1,NaN,NaN,NaN,-1.0,0.0,5.0,-1.0,0.0,5.0,NaN,NaN,NaN,0.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-1.0,0.0,5.0,0.0,0.0,False,2.0
4,1,2013-04-04 08:32:15,12,187,NaN,NaN,219,29604,4,3.5,1,2.64,0.1241,4.93,4.0,143.58,0,23246,1,0,4,0,1,1,NaN,1,NaN,NaN,NaN,0.0,0.0,NaN,0.0,0.0,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN,0.0,0.0,False,4.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9917525,665572,2013-05-21 11:06:37,24,216,NaN,NaN,117,32019,4,3.5,0,2.48,0.0551,4.53,NaN,66.07,0,19246,2,7,1,0,1,0,NaN,0,NaN,NaN,NaN,1.0,0.0,22.0,1.0,0.0,127.0,-1.0,0.0,27.0,1.0,0.0,22.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True,4.0
9917526,665572,2013-05-21 11:06:37,24,216,NaN,NaN,117,33959,4,3.0,1,2.20,0.3344,4.39,NaN,67.10,0,19246,2,7,1,0,1,0,NaN,0,NaN,NaN,NaN,0.0,0.0,NaN,0.0,0.0,NaN,0.0,0.0,16.0,1.0,0.0,22.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True,4.0
9917527,665572,2013-05-21 11:06:37,24,216,NaN,NaN,117,35240,4,NaN,0,1.79,NaN,4.64,NaN,73.91,0,19246,2,7,1,0,1,0,NaN,0,NaN,NaN,NaN,1.0,0.0,55.0,0.0,0.0,NaN,0.0,0.0,16.0,0.0,0.0,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True,4.0
9917528,665572,2013-05-21 11:06:37,24,216,NaN,NaN,117,94437,4,NaN,0,2.94,0.0928,4.64,NaN,66.07,0,19246,2,7,1,0,1,0,NaN,0,NaN,NaN,NaN,1.0,0.0,43.0,1.0,0.0,43.0,-1.0,0.0,12.0,-1.0,0.0,12.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True,4.0


# date_time

In [20]:
def convert_date(df):
    df['date_time'] = pd.to_datetime(df['date_time'])
    df['booking_week'] = df['date_time'].dt.isocalendar().week
    df['booking_month'] = df['date_time'].dt.month
    df['booking_dayofyear'] = df['date_time'].dt.dayofyear
    df['booking_dayofweek'] = df['date_time'].dt.dayofweek
    midstay = df['date_time'] + pd.to_timedelta(df.srch_booking_window, unit='days') + pd.to_timedelta(df.srch_length_of_stay//2, unit='days')
    df['midstay_week'] = midstay.dt.isocalendar().week
    df['midstay_month'] = midstay.dt.month
    df['midstay_dayofyear'] = midstay.dt.dayofyear
    df['midstay_dayofweek'] = midstay.dt.dayofweek
    df.drop(columns='date_time', inplace=True)

convert_date(df)

In [21]:
df

,srch_id,site_id,visitor_location_country_id,visitor_hist_starrating,visitor_hist_adr_usd,prop_country_id,prop_id,prop_starrating,prop_review_score,prop_brand_bool,prop_location_score1,prop_location_score2,prop_log_historical_price,position,price_usd,promotion_flag,srch_destination_id,srch_length_of_stay,srch_booking_window,srch_adults_count,srch_children_count,srch_room_count,srch_saturday_night_bool,srch_query_affinity_score,random_bool,comp1_rate,comp1_inv,comp1_rate_percent_diff,comp2_rate,comp2_inv,comp2_rate_percent_diff,comp3_rate,comp3_inv,comp3_rate_percent_diff,comp4_rate,comp4_inv,comp4_rate_percent_diff,comp5_rate,comp5_inv,comp5_rate_percent_diff,comp6_rate,comp6_inv,comp6_rate_percent_diff,comp7_rate,comp7_inv,comp7_rate_percent_diff,comp8_rate,comp8_inv,comp8_rate_percent_diff,click_bool,booking_bool,is_test,prop_starrating_w0,booking_week,booking_month,booking_dayofyear,booking_dayofweek,midstay_week,midstay_month,midstay_dayofyear,midstay_dayofweek
0,1,12,187,NaN,NaN,219,893,3,3.5,1,2.83,0.0438,4.95,27.0,104.77,0,23246,1,0,4,0,1,1,NaN,1,NaN,NaN,NaN,0.0,0.0,NaN,0.0,0.0,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN,0.0,0.0,False,3.0,14,4,94,3,14,4,94,3
1,1,12,187,NaN,NaN,219,10404,4,4.0,1,2.20,0.0149,5.03,26.0,170.74,0,23246,1,0,4,0,1,1,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,0.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN,0.0,0.0,False,4.0,14,4,94,3,14,4,94,3
2,1,12,187,NaN,NaN,219,21315,3,4.5,1,2.20,0.0245,4.92,21.0,179.80,0,23246,1,0,4,0,1,1,NaN,1,NaN,NaN,NaN,0.0,0.0,NaN,0.0,0.0,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN,0.0,0.0,False,3.0,14,4,94,3,14,4,94,3
3,1,12,187,NaN,NaN,219,27348,2,4.0,1,2.83,0.0125,4.39,34.0,602.77,0,23246,1,0,4,0,1,1,NaN,1,NaN,NaN,NaN,-1.0,0.0,5.0,-1.0,0.0,5.0,NaN,NaN,NaN,0.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-1.0,0.0,5.0,0.0,0.0,False,2.0,14,4,94,3,14,4,94,3
4,1,12,187,NaN,NaN,219,29604,4,3.5,1,2.64,0.1241,4.93,4.0,143.58,0,23246,1,0,4,0,1,1,NaN,1,NaN,NaN,NaN,0.0,0.0,NaN,0.0,0.0,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN,0.0,0.0,False,4.0,14,4,94,3,14,4,94,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9917525,665572,24,216,NaN,NaN,117,32019,4,3.5,0,2.48,0.0551,4.53,NaN,66.07,0,19246,2,7,1,0,1,0,NaN,0,NaN,NaN,NaN,1.0,0.0,22.0,1.0,0.0,127.0,-1.0,0.0,27.0,1.0,0.0,22.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True,4.0,21,5,141,1,22,5,149,2
9917526,665572,24,216,NaN,NaN,117,33959,4,3.0,1,2.20,0.3344,4.39,NaN,67.10,0,19246,2,7,1,0,1,0,NaN,0,NaN,NaN,NaN,0.0,0.0,NaN,0.0,0.0,NaN,0.0,0.0,16.0,1.0,0.0,22.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True,4.0,21,5,141,1,22,5,149,2
9917527,665572,24,216,NaN,NaN,117,35240,4,NaN,0,1.79,NaN,4.64,NaN,73.91,0,19246,2,7,1,0,1,0,NaN,0,NaN,NaN,NaN,1.0,0.0,55.0,0.0,0.0,NaN,0.0,0.0,16.0,0.0,0.0,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True,4.0,21,5,141,1,22,5,149,2
9917528,665572,24,216,NaN,NaN,117,94437,4,NaN,0,2.94,0.0928,4.64,NaN,66.07,0,19246,2,7,1,0,1,0,NaN,0,NaN,NaN,NaN,1.0,0.0,43.0,1.0,0.0,43.0,-1.0,0.0,12.0,-1.0,0.0,12.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True,4.0,21,5,141,1,22,5,149,2


In [22]:
# TODO:
# get a smart booking_period_congestion (per prop, dest)
# for each property and destination get a day-of-year load schedule
# for each search period do a sum over the booked days, this is the congestion of the booking

# Numerical column normalization

In [23]:
# get average price per property - not a good idea, price can be total or per-room, with or without taxes
# get average price per destination, prop_starrating - not a good idea, price can be total or per-room, with or without taxes
# get average price per destination, prop_starrating, prop_review_score - same
# get average price per destination - same

## prop_starrating

In [24]:
def to_list(v):
    if hasattr(v, '__iter__') and type(v) != str:
        return v
    else:
        return [v]

In [25]:
def make_aggregate_feature(df, groupby, col, transforms='z', use_median=False, filter_query=None):
    gl = to_list(groupby)
    transforms = to_list(transforms)
    l = list(set(gl + ['srch_id', 'prop_id', col]))
    tmp = df.query(filter_query) if filter_query else df
    tmp = tmp[l].groupby(groupby).agg(
        col_avg = (col, 'mean'),
        col_med = (col, 'median'),
        col_std = (col, 'std'),
    )
    print(f'{tmp.col_std.isna().sum() / len(tmp):.2} NAs')

    tmp1 = df[l].set_index(gl)
    tmp1 = tmp1.join(tmp, how='left')
    res_cols = []
    for transform in transforms:
        if transform == 'z':
            res_col = f'{col}_z_{"__".join(gl)}'
            tmp1[res_col] = (tmp1[col] - (tmp1.col_med if use_median else tmp1.col_avg)) / tmp1.col_std
        elif transform == 'log_diff':
            res_col = f'{col}_ld_{"__".join(gl)}'
            tmp1[res_col] = np.log(tmp1[col]) - np.log(tmp1.col_med if use_median else tmp1.col_avg)
        elif transform == 'diff':
            res_col = f'{col}_d_{"__".join(gl)}'
            tmp1[res_col] = tmp1[col] - (tmp1.col_med if use_median else tmp1.col_avg)
        elif transform == 'med':
            res_col = f'{col}_{transform}_{"__".join(gl)}'
            tmp1[res_col] = tmp1.col_med
        elif transform == 'avg':
            res_col = f'{col}_{transform}_{"__".join(gl)}'
            tmp1[res_col] = tmp1.col_avg
        elif transform == 'std':
            res_col = f'{col}_{transform}_{"__".join(gl)}'
            tmp1[res_col] = tmp1.col_std
        else:
            assert False, f'unsupported transform "{transform}"'
        res_cols.append(res_col)
    tmp1 = tmp1.reset_index().set_index(['srch_id', 'prop_id']).sort_index()
    return tmp1.drop(columns=[c for c in tmp1.columns if c not in res_cols])

In [26]:
# the average property star rating per destination
tosub_prop_starrating_per_dest = make_aggregate_feature(df, groupby ='srch_destination_id', col ='prop_starrating_w0', transforms=['std', 'med', 'diff'], use_median=True)
tosub_prop_starrating_per_dest

0.0023 NAs


prop_starrating_w0_std_srch_destination_id   
srch_id prop_id                                               
1       893                                        0.808048  \
        10404                                      0.808048   
        21315                                      0.808048   
        27348                                      0.808048   
        29604                                      0.808048   
...                                                     ...   
665572  32019                                      0.694416   
        33959                                      0.694416   
        35240                                      0.694416   
        94437                                      0.694416   
        99509                                      0.694416   

                 prop_starrating_w0_med_srch_destination_id   
srch_id prop_id                                               
1       893                                             3.0  \
        10404                                           3.0   
        21315                                           3.0   
        27348                                           3.0   
        29604                                           3.0   
...                                                     ...   
665572  32019                                           4.0   
        33959                                           4.0   
        35240                                           4.0   
        94437                                           4.0   
        99509                                           4.0   

                 prop_starrating_w0_d_srch_destination_id  
srch_id prop_id                                            
1       893                                           0.0  
        10404                                         1.0  
        21315                                         0.0  
        27348                                        -1.0  
        29604                                         1.0  
...                                                   ...  
665572  32019                                         0.0  
        33959                                         0.0  
        35240                                         0.0  
        94437                                         0.0  
        99509                                         0.0  

[9917530 rows x 3 columns]

In [27]:
norm_prop_starrating_per_srch = make_aggregate_feature(df, groupby ='srch_id', col ='prop_starrating_w0', transforms=['std', 'med', 'diff'], use_median=True)
norm_prop_starrating_per_srch

0.00096 NAs


prop_starrating_w0_std_srch_id   
srch_id prop_id                                   
1       893                            0.766356  \
        10404                          0.766356   
        21315                          0.766356   
        27348                          0.766356   
        29604                          0.766356   
...                                         ...   
665572  32019                          0.377964   
        33959                          0.377964   
        35240                          0.377964   
        94437                          0.377964   
        99509                          0.377964   

                 prop_starrating_w0_med_srch_id  prop_starrating_w0_d_srch_id  
srch_id prop_id                                                                
1       893                                 3.0                           0.0  
        10404                               3.0                           1.0  
        21315                               3.0                           0.0  
        27348                               3.0                          -1.0  
        29604                               3.0                           1.0  
...                                         ...                           ...  
665572  32019                               4.0                           0.0  
        33959                               4.0                           0.0  
        35240                               4.0                           0.0  
        94437                               4.0                           0.0  
        99509                               4.0                           0.0  

[9917530 rows x 3 columns]

In [28]:
df['hist_starrating_diff'] = df['visitor_hist_starrating'] - df['prop_starrating']

## prop_review_score

In [29]:
tosub_prop_review_score_per_dest = make_aggregate_feature(df, groupby = ['srch_destination_id'], col ='prop_review_score',
                                                          transforms=['std', 'med', 'diff'], use_median=True)
tosub_prop_review_score_per_dest

0.0051 NAs


prop_review_score_std_srch_destination_id   
srch_id prop_id                                              
1       893                                       0.527944  \
        10404                                     0.527944   
        21315                                     0.527944   
        27348                                     0.527944   
        29604                                     0.527944   
...                                                    ...   
665572  32019                                     0.694472   
        33959                                     0.694472   
        35240                                     0.694472   
        94437                                     0.694472   
        99509                                     0.694472   

                 prop_review_score_med_srch_destination_id   
srch_id prop_id                                              
1       893                                            4.0  \
        10404                                          4.0   
        21315                                          4.0   
        27348                                          4.0   
        29604                                          4.0   
...                                                    ...   
665572  32019                                          4.0   
        33959                                          4.0   
        35240                                          4.0   
        94437                                          4.0   
        99509                                          4.0   

                 prop_review_score_d_srch_destination_id  
srch_id prop_id                                           
1       893                                         -0.5  
        10404                                        0.0  
        21315                                        0.5  
        27348                                        0.0  
        29604                                       -0.5  
...                                                  ...  
665572  32019                                       -0.5  
        33959                                       -1.0  
        35240                                        NaN  
        94437                                        NaN  
        99509                                        0.5  

[9917530 rows x 3 columns]

In [30]:
tosub_prop_review_score_per_dest_prop_starrating = make_aggregate_feature(df, groupby = ['srch_destination_id', 'prop_starrating'], col ='prop_review_score',
                                                                          transforms=['std', 'med', 'diff'], use_median=True)
tosub_prop_review_score_per_dest_prop_starrating

0.16 NAs


prop_review_score_std_srch_destination_id__prop_starrating   
srch_id prop_id                                                               
1       893                                               0.461749           \
        10404                                             0.349965            
        21315                                             0.461749            
        27348                                             0.499591            
        29604                                             0.349965            
...                                                            ...            
665572  32019                                             0.745356            
        33959                                             0.745356            
        35240                                             0.745356            
        94437                                             0.745356            
        99509                                             0.745356            

                 prop_review_score_med_srch_destination_id__prop_starrating   
srch_id prop_id                                                               
1       893                                                    4.0           \
        10404                                                  4.0            
        21315                                                  4.0            
        27348                                                  3.5            
        29604                                                  4.0            
...                                                            ...            
665572  32019                                                  4.0            
        33959                                                  4.0            
        35240                                                  4.0            
        94437                                                  4.0            
        99509                                                  4.0            

                 prop_review_score_d_srch_destination_id__prop_starrating  
srch_id prop_id                                                            
1       893                                                   -0.5         
        10404                                                  0.0         
        21315                                                  0.5         
        27348                                                  0.5         
        29604                                                 -0.5         
...                                                            ...         
665572  32019                                                 -0.5         
        33959                                                 -1.0         
        35240                                                  NaN         
        94437                                                  NaN         
        99509                                                  0.5         

[9917530 rows x 3 columns]

In [31]:
norm_prop_review_score_per_srch = make_aggregate_feature(df, groupby ='srch_id', col ='prop_review_score',
                                                         transforms=['std', 'med', 'diff'], use_median=True)
norm_prop_review_score_per_srch

0.00086 NAs


prop_review_score_std_srch_id  prop_review_score_med_srch_id   
srch_id prop_id                                                                 
1       893                           0.533854                           3.75  \
        10404                         0.533854                           3.75   
        21315                         0.533854                           3.75   
        27348                         0.533854                           3.75   
        29604                         0.533854                           3.75   
...                                        ...                            ...   
665572  32019                         0.821584                           4.50   
        33959                         0.821584                           4.50   
        35240                         0.821584                           4.50   
        94437                         0.821584                           4.50   
        99509                         0.821584                           4.50   

                 prop_review_score_d_srch_id  
srch_id prop_id                               
1       893                            -0.25  
        10404                           0.25  
        21315                           0.75  
        27348                           0.25  
        29604                          -0.25  
...                                      ...  
665572  32019                          -1.00  
        33959                          -1.50  
        35240                            NaN  
        94437                            NaN  
        99509                           0.00  

[9917530 rows x 3 columns]

## prop_location_score1

In [32]:
tosub_prop_location_score1_per_dest = make_aggregate_feature(df, groupby = 'srch_destination_id', col ='prop_location_score1',
                                                             transforms=['std', 'med', 'diff'], use_median=True)
tosub_prop_location_score1_per_dest

0.0 NAs


prop_location_score1_std_srch_destination_id   
srch_id prop_id                                                 
1       893                                          0.667135  \
        10404                                        0.667135   
        21315                                        0.667135   
        27348                                        0.667135   
        29604                                        0.667135   
...                                                       ...   
665572  32019                                        0.841209   
        33959                                        0.841209   
        35240                                        0.841209   
        94437                                        0.841209   
        99509                                        0.841209   

                 prop_location_score1_med_srch_destination_id   
srch_id prop_id                                                 
1       893                                               2.3  \
        10404                                             2.3   
        21315                                             2.3   
        27348                                             2.3   
        29604                                             2.3   
...                                                       ...   
665572  32019                                             2.3   
        33959                                             2.3   
        35240                                             2.3   
        94437                                             2.3   
        99509                                             2.3   

                 prop_location_score1_d_srch_destination_id  
srch_id prop_id                                              
1       893                                            0.53  
        10404                                         -0.10  
        21315                                         -0.10  
        27348                                          0.53  
        29604                                          0.34  
...                                                     ...  
665572  32019                                          0.18  
        33959                                         -0.10  
        35240                                         -0.51  
        94437                                          0.64  
        99509                                         -0.22  

[9917530 rows x 3 columns]

In [33]:
tosub_prop_location_score1_per_dest_prop_starrating = make_aggregate_feature(df, groupby = ['srch_destination_id', 'prop_starrating'], col ='prop_location_score1',
                                                                             transforms=['std', 'med', 'diff'], use_median=True)
tosub_prop_location_score1_per_dest_prop_starrating

0.1 NAs


prop_location_score1_std_srch_destination_id__prop_starrating   
srch_id prop_id                                                                  
1       893                                               0.578988              \
        10404                                             0.637440               
        21315                                             0.578988               
        27348                                             0.766894               
        29604                                             0.637440               
...                                                            ...               
665572  32019                                             0.687043               
        33959                                             0.687043               
        35240                                             0.687043               
        94437                                             0.687043               
        99509                                             0.687043               

                 prop_location_score1_med_srch_destination_id__prop_starrating   
srch_id prop_id                                                                  
1       893                                                   2.20              \
        10404                                                 2.64               
        21315                                                 2.20               
        27348                                                 2.20               
        29604                                                 2.64               
...                                                            ...               
665572  32019                                                 2.48               
        33959                                                 2.48               
        35240                                                 2.48               
        94437                                                 2.48               
        99509                                                 2.48               

                 prop_location_score1_d_srch_destination_id__prop_starrating  
srch_id prop_id                                                               
1       893                                                   0.63            
        10404                                                -0.44            
        21315                                                 0.00            
        27348                                                 0.63            
        29604                                                 0.00            
...                                                            ...            
665572  32019                                                 0.00            
        33959                                                -0.28            
        35240                                                -0.69            
        94437                                                 0.46            
        99509                                                -0.40            

[9917530 rows x 3 columns]

In [34]:
# do multiple srch_ids correspond to one srch_destination_id?
df.groupby('srch_destination_id').srch_id.nunique() # => yes

srch_destination_id
2        3
3        1
5        2
6        2
7        1
        ..
28412    1
28413    1
28414    1
28415    1
28416    5
Name: srch_id, Length: 23715, dtype: int64

In [35]:
norm_prop_location_score1_per_srch = make_aggregate_feature(df, groupby=['srch_id'], col='prop_location_score1',
                                                            transforms=['std', 'med', 'diff'], use_median=True)
norm_prop_location_score1_per_srch

0.0 NAs


prop_location_score1_std_srch_id   
srch_id prop_id                                     
1       893                              0.518734  \
        10404                            0.518734   
        21315                            0.518734   
        27348                            0.518734   
        29604                            0.518734   
...                                           ...   
665572  32019                            0.369498   
        33959                            0.369498   
        35240                            0.369498   
        94437                            0.369498   
        99509                            0.369498   

                 prop_location_score1_med_srch_id   
srch_id prop_id                                     
1       893                                   2.3  \
        10404                                 2.3   
        21315                                 2.3   
        27348                                 2.3   
        29604                                 2.3   
...                                           ...   
665572  32019                                 2.3   
        33959                                 2.3   
        35240                                 2.3   
        94437                                 2.3   
        99509                                 2.3   

                 prop_location_score1_d_srch_id  
srch_id prop_id                                  
1       893                                0.53  
        10404                             -0.10  
        21315                             -0.10  
        27348                              0.53  
        29604                              0.34  
...                                         ...  
665572  32019                              0.18  
        33959                             -0.10  
        35240                             -0.51  
        94437                              0.64  
        99509                             -0.22  

[9917530 rows x 3 columns]

In [36]:
#make_aggregate_feature(groupby=['srch_id', 'prop_starrating'], col='prop_location_score1')

## prop_location_score2

In [37]:
tosub_prop_location_score2_per_dest_id = make_aggregate_feature(df, groupby = ['srch_destination_id'], col ='prop_location_score2',
                                                                transforms=['std', 'med', 'diff'], use_median=True)
tosub_prop_location_score2_per_dest_id

0.26 NAs


prop_location_score2_std_srch_destination_id   
srch_id prop_id                                                 
1       893                                          0.043762  \
        10404                                        0.043762   
        21315                                        0.043762   
        27348                                        0.043762   
        29604                                        0.043762   
...                                                       ...   
665572  32019                                        0.115111   
        33959                                        0.115111   
        35240                                        0.115111   
        94437                                        0.115111   
        99509                                        0.115111   

                 prop_location_score2_med_srch_destination_id   
srch_id prop_id                                                 
1       893                                            0.0164  \
        10404                                          0.0164   
        21315                                          0.0164   
        27348                                          0.0164   
        29604                                          0.0164   
...                                                       ...   
665572  32019                                          0.0879   
        33959                                          0.0879   
        35240                                          0.0879   
        94437                                          0.0879   
        99509                                          0.0879   

                 prop_location_score2_d_srch_destination_id  
srch_id prop_id                                              
1       893                                          0.0274  
        10404                                       -0.0015  
        21315                                        0.0081  
        27348                                       -0.0039  
        29604                                        0.1077  
...                                                     ...  
665572  32019                                       -0.0328  
        33959                                        0.2465  
        35240                                           NaN  
        94437                                        0.0049  
        99509                                       -0.0535  

[9917530 rows x 3 columns]

In [38]:
tosub_prop_location_score2_per_dest_id_prop_starrating = make_aggregate_feature(df, groupby=['srch_destination_id', 'prop_starrating'], col='prop_location_score2',
                                                                                transforms=['std', 'med', 'diff'], use_median=True)
tosub_prop_location_score2_per_dest_id_prop_starrating

0.52 NAs


prop_location_score2_std_srch_destination_id__prop_starrating   
srch_id prop_id                                                                  
1       893                                               0.030738              \
        10404                                             0.053266               
        21315                                             0.030738               
        27348                                             0.021450               
        29604                                             0.053266               
...                                                            ...               
665572  32019                                             0.130403               
        33959                                             0.130403               
        35240                                             0.130403               
        94437                                             0.130403               
        99509                                             0.130403               

                 prop_location_score2_med_srch_destination_id__prop_starrating   
srch_id prop_id                                                                  
1       893                                                 0.0164              \
        10404                                               0.0660               
        21315                                               0.0164               
        27348                                               0.0125               
        29604                                               0.0660               
...                                                            ...               
665572  32019                                               0.0551               
        33959                                               0.0551               
        35240                                               0.0551               
        94437                                               0.0551               
        99509                                               0.0551               

                 prop_location_score2_d_srch_destination_id__prop_starrating  
srch_id prop_id                                                               
1       893                                                 0.0274            
        10404                                              -0.0511            
        21315                                               0.0081            
        27348                                               0.0000            
        29604                                               0.0581            
...                                                            ...            
665572  32019                                               0.0000            
        33959                                               0.2793            
        35240                                                  NaN            
        94437                                               0.0377            
        99509                                              -0.0207            

[9917530 rows x 3 columns]

In [39]:
norm_prop_location_score2_per_srch = make_aggregate_feature(df, groupby=['srch_id'], col='prop_location_score2',
                                                            transforms=['std', 'med', 'diff'], use_median=True)
norm_prop_location_score2_per_srch

0.05 NAs


prop_location_score2_std_srch_id   
srch_id prop_id                                     
1       893                              0.047217  \
        10404                            0.047217   
        21315                            0.047217   
        27348                            0.047217   
        29604                            0.047217   
...                                           ...   
665572  32019                            0.116369   
        33959                            0.116369   
        35240                            0.116369   
        94437                            0.116369   
        99509                            0.116369   

                 prop_location_score2_med_srch_id   
srch_id prop_id                                     
1       893                               0.03005  \
        10404                             0.03005   
        21315                             0.03005   
        27348                             0.03005   
        29604                             0.03005   
...                                           ...   
665572  32019                             0.07395   
        33959                             0.07395   
        35240                             0.07395   
        94437                             0.07395   
        99509                             0.07395   

                 prop_location_score2_d_srch_id  
srch_id prop_id                                  
1       893                             0.01375  
        10404                          -0.01515  
        21315                          -0.00555  
        27348                          -0.01755  
        29604                           0.09405  
...                                         ...  
665572  32019                          -0.01885  
        33959                           0.26045  
        35240                               NaN  
        94437                           0.01885  
        99509                          -0.03955  

[9917530 rows x 3 columns]

## prop_log_historical_price

In [40]:
(df.prop_log_historical_price==0).sum() / len(df)

0.14423218281164765

In [41]:
# number of booked properties
df_train.query('booking_bool==1').prop_id.nunique() / df_train.prop_id.nunique()

0.336356524904541

In [42]:
prop_with_0hist = df_train[df_train.prop_log_historical_price==0].prop_id.unique()
df_train[df_train.prop_id.isin(prop_with_0hist)].groupby(['prop_id', 'prop_log_historical_price', 'price_usd']).booking_bool.sum()

prop_id  prop_log_historical_price  price_usd
1        0.00                       90.00        0
                                    95.00        0
                                    99.00        0
                                    99.24        0
                                    108.00       0
                                                ..
140820   5.29                       108.62       0
                                    151.89       0
                                    169.00       0
         5.35                       206.14       0
         5.37                       228.88       0
Name: booking_bool, Length: 3459455, dtype: int64

In [43]:
# prop_log_historical_price has multiple value (changing over time). 0 has a special meaning so we set to to NA
df.loc[df.prop_log_historical_price==0, 'prop_log_historical_price'] = pd.NA

## price_usd

The price_usd is the displayed price of the hotel for the given search. Note that different countries have different conventions regarding displaying taxes and fees and the value may be per night or for the whole stay

Note: this implies that it's not really correct to normalize price across anything other than search_id without first transforming it to price per night (even so, the taxes are not necessarily accounted for). As such, it's not really comparable with the historical price either.

In [44]:
#top_grp = 'site_id'
top_grp = 'visitor_location_country_id'
next_grp = 'srch_length_of_stay'

tmp = df.groupby([top_grp, 'prop_id', next_grp]).price_usd.mean().reset_index(level=2)#.pivot(columns=next_grp, values='price_usd')
tmp['price_usd'] = tmp['price_usd'] / tmp[next_grp]
#tmp[next_grp] = tmp[next_grp].astype(str)
tmp = tmp.pivot(columns=next_grp, values='price_usd')
tmp[(tmp[1].notna() & tmp[2].notna())].groupby(top_grp).mean()
# # group tmp by 'visitor_location_country_id', 'prop_id' and calculate the grouped correlation between the price and room count column
# #tmp.groupby(['visitor_location_country_id', 'prop_id']).apply(lambda x: x['price_usd'].corr(x[next_grp]))
#tmp = tmp.groupby([top_grp, 'prop_id']).agg({'price_usd': ['mean', 'std']})
# tmp['norm_std'] = tmp['price_usd']['std'] / tmp['price_usd']['mean']
# tmp.groupby(top_grp).norm_std.mean()

srch_length_of_stay,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,38,40,41,44,57,58,59
visitor_location_country_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,59.375000,32.181667,22.330000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,199.650178,97.510329,57.057176,41.406863,22.626087,19.000000,60.214286,NaN,NaN,NaN,NaN,NaN,NaN,10.571429,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,164.786057,82.430379,47.566899,37.675605,33.746329,20.834220,9.260857,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,207.745553,107.514155,108.250000,1304.550595,70.342800,NaN,NaN,16.160714,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,269.908737,109.572218,94.937708,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
221,101.534884,74.040233,45.166667,20.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
223,143.116949,70.031907,39.283333,72.583333,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
224,56.904561,36.388158,25.650185,12.554167,NaN,NaN,NaN,4.299125,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [45]:
# A zero price is nonsensical
df.loc[df.price_usd==0, 'price_usd'] = pd.NA
df['price_usd'] = np.log(df['price_usd'])

In [46]:
# relative to the historical price (difference of logs is the same price_usd/exp(prop_log_historical_price) - negative bad, positive good
price_hist_logdiff = df['price_usd'] - df['prop_log_historical_price']
df['price_hist_logdiff'] = price_hist_logdiff

In [47]:
# how good of a deal this is relative to others in the search query
norm_prop_price_hist_advantage_per_srch_id = make_aggregate_feature(df, ['srch_id'], 'price_hist_logdiff',
                                                                    transforms=['std', 'med', 'diff'], use_median=True)
norm_prop_price_hist_advantage_per_srch_id

0.14 NAs


price_hist_logdiff_std_srch_id   
srch_id prop_id                                   
1       893                            0.415762  \
        10404                          0.415762   
        21315                          0.415762   
        27348                          0.415762   
        29604                          0.415762   
...                                         ...   
665572  32019                          0.092774   
        33959                          0.092774   
        35240                          0.092774   
        94437                          0.092774   
        99509                          0.092774   

                 price_hist_logdiff_med_srch_id  price_hist_logdiff_d_srch_id  
srch_id prop_id                                                                
1       893                            0.087924                     -0.386156  
        10404                          0.087924                      0.022218  
        21315                          0.087924                      0.183921  
        27348                          0.087924                      1.923612  
        29604                          0.087924                     -0.051031  
...                                         ...                           ...  
665572  32019                         -0.337152                     -0.002133  
        33959                         -0.337152                      0.153336  
        35240                         -0.337152                      0.000000  
        94437                         -0.337152                     -0.112133  
        99509                         -0.337152                      0.104603  

[9917530 rows x 3 columns]

In [48]:
df.loc[df.visitor_hist_adr_usd==0, 'visitor_hist_adr_usd'] = pd.NA
df['visitor_hist_adr_usd_logdiff'] = df['price_usd'] - np.log(df['visitor_hist_adr_usd'])

In [49]:
norm_visitor_price_hist_advantage_per_srch_id = make_aggregate_feature(df, ['srch_id'], 'visitor_hist_adr_usd_logdiff', transforms='diff', use_median=True)
norm_visitor_price_hist_advantage_per_srch_id


0.95 NAs


visitor_hist_adr_usd_logdiff_d_srch_id
srch_id prop_id                                        
1       893                                         NaN
        10404                                       NaN
        21315                                       NaN
        27348                                       NaN
        29604                                       NaN
...                                                 ...
665572  32019                                       NaN
        33959                                       NaN
        35240                                       NaN
        94437                                       NaN
        99509                                       NaN

[9917530 rows x 1 columns]

In [50]:
norm_price_per_srch = make_aggregate_feature(df, ['srch_id'], 'price_usd',
                                             transforms=['std', 'med', 'diff'], use_median=True)
norm_price_per_srch

0.0 NAs


price_usd_std_srch_id  price_usd_med_srch_id   
srch_id prop_id                                                 
1       893                   0.390255               4.934823  \
        10404                 0.390255               4.934823   
        21315                 0.390255               4.934823   
        27348                 0.390255               4.934823   
        29604                 0.390255               4.934823   
...                                ...                    ...   
665572  32019                 0.183529               4.249209   
        33959                 0.183529               4.249209   
        35240                 0.183529               4.249209   
        94437                 0.183529               4.249209   
        99509                 0.183529               4.249209   

                 price_usd_d_srch_id  
srch_id prop_id                       
1       893                -0.283055  
        10404               0.205319  
        21315               0.257022  
        27348               1.466713  
        29604               0.032070  
...                              ...  
665572  32019              -0.058494  
        33959              -0.043025  
        35240               0.053639  
        94437              -0.058494  
        99509               0.158241  

[9917530 rows x 3 columns]

In [51]:
norm_price_per_search_prop_starrating = make_aggregate_feature(df, ['srch_id', 'prop_starrating'], 'price_usd',
                                                               transforms=['std', 'med', 'diff'], use_median=True)
norm_price_per_search_prop_starrating

0.16 NAs


price_usd_std_srch_id__prop_starrating   
srch_id prop_id                                           
1       893                                    0.252016  \
        10404                                  0.256295   
        21315                                  0.252016   
        27348                                  0.672170   
        29604                                  0.256295   
...                                                 ...   
665572  32019                                  0.085045   
        33959                                  0.085045   
        35240                                  0.085045   
        94437                                  0.085045   
        99509                                  0.085045   

                 price_usd_med_srch_id__prop_starrating   
srch_id prop_id                                           
1       893                                    4.930148  \
        10404                                  5.028017   
        21315                                  4.930148   
        27348                                  4.651767   
        29604                                  5.028017   
...                                                 ...   
665572  32019                                  4.227697   
        33959                                  4.227697   
        35240                                  4.227697   
        94437                                  4.227697   
        99509                                  4.227697   

                 price_usd_d_srch_id__prop_starrating  
srch_id prop_id                                        
1       893                                 -0.278381  
        10404                                0.112125  
        21315                                0.261697  
        27348                                1.749768  
        29604                               -0.061124  
...                                               ...  
665572  32019                               -0.036982  
        33959                               -0.021513  
        35240                                0.075151  
        94437                               -0.036982  
        99509                                0.179754  

[9917530 rows x 3 columns]

# country

In [52]:
df['same_country'] = (df['prop_country_id'] == df['visitor_location_country_id'])

# srch_query_affinity_score

In [53]:
norm_srch_query_affinity_score = make_aggregate_feature(df, ['srch_id'], 'srch_query_affinity_score',
                                                        transforms=['std', 'diff'], use_median=True)

0.93 NAs


In [54]:
# number of search results - per search id
df = df.merge(df.groupby('srch_id').agg(srch_res_count = ('srch_id', 'count')), on='srch_id')

In [55]:
# numer of properties per search destination id
df = df.merge(df.groupby('srch_destination_id').agg(prop_count_per_srch_dest = ('prop_id', 'count')), on='srch_destination_id')

In [56]:
# number of search_destination_ids per property
df = df.merge(df.groupby('prop_id').agg(srch_dest_count_per_prop = ('srch_destination_id', 'count')), on='prop_id')

In [57]:
df

,srch_id,site_id,visitor_location_country_id,visitor_hist_starrating,visitor_hist_adr_usd,prop_country_id,prop_id,prop_starrating,prop_review_score,prop_brand_bool,prop_location_score1,prop_location_score2,prop_log_historical_price,position,price_usd,promotion_flag,srch_destination_id,srch_length_of_stay,srch_booking_window,srch_adults_count,srch_children_count,srch_room_count,srch_saturday_night_bool,srch_query_affinity_score,random_bool,comp1_rate,comp1_inv,comp1_rate_percent_diff,comp2_rate,comp2_inv,comp2_rate_percent_diff,comp3_rate,comp3_inv,comp3_rate_percent_diff,comp4_rate,comp4_inv,comp4_rate_percent_diff,comp5_rate,comp5_inv,comp5_rate_percent_diff,comp6_rate,comp6_inv,comp6_rate_percent_diff,comp7_rate,comp7_inv,comp7_rate_percent_diff,comp8_rate,comp8_inv,comp8_rate_percent_diff,click_bool,booking_bool,is_test,prop_starrating_w0,booking_week,booking_month,booking_dayofyear,booking_dayofweek,midstay_week,midstay_month,midstay_dayofyear,midstay_dayofweek,hist_starrating_diff,price_hist_logdiff,visitor_hist_adr_usd_logdiff,same_country,srch_res_count,prop_count_per_srch_dest,srch_dest_count_per_prop
0,1,12,187,NaN,NaN,219,893,3,3.5,1,2.83,0.0438,4.95,27.0,4.651767,0,23246,1,0,4,0,1,1,NaN,1,NaN,NaN,NaN,0.0,0.0,NaN,0.0,0.0,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN,0.0,0.0,False,3.0,14,4,94,3,14,4,94,3,NaN,-0.298233,NaN,False,28,27173,1237
1,896,5,2,NaN,NaN,219,893,3,3.5,1,2.83,0.0438,4.98,24.0,4.836282,0,23246,1,16,1,0,1,1,NaN,0,NaN,NaN,NaN,0.0,0.0,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN,0.0,0.0,False,3.0,19,5,129,3,21,5,145,5,NaN,-0.143718,NaN,False,36,27173,1237
2,1495,5,219,NaN,NaN,219,893,3,3.5,1,2.83,0.0438,4.98,28.0,4.934474,0,23246,1,40,1,0,1,0,NaN,0,NaN,NaN,NaN,0.0,0.0,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,False,3.0,17,4,114,2,23,6,154,0,NaN,-0.045526,NaN,True,34,27173,1237
3,2052,5,219,NaN,NaN,219,893,3,3.5,1,2.83,0.0438,4.96,29.0,4.691348,0,23246,1,0,2,0,1,1,NaN,0,NaN,NaN,NaN,0.0,0.0,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-1.0,0.0,6.0,0.0,0.0,False,3.0,2,1,11,4,2,1,11,4,NaN,-0.268652,NaN,True,34,27173,1237
4,2056,5,219,NaN,NaN,219,893,3,3.5,1,2.83,0.0438,4.95,30.0,4.553877,0,23246,1,1,2,0,1,1,NaN,0,NaN,NaN,NaN,0.0,0.0,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN,0.0,0.0,False,3.0,10,3,66,3,10,3,67,4,NaN,-0.396123,NaN,True,35,27173,1237
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9917525,661574,5,109,NaN,NaN,109,120227,2,NaN,0,2.77,NaN,NaN,NaN,3.931826,0,306,1,122,2,0,1,1,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True,2.0,14,4,91,0,31,8,213,3,NaN,NaN,NaN,True,27,27,1
9917526,662118,15,55,NaN,NaN,109,60440,4,3.5,0,0.00,0.0000,5.13,NaN,4.846939,1,18400,6,121,2,0,1,0,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True,4.0,18,4,120,1,35,9,244,6,NaN,-0.283061,NaN,False,11,11,1
9917527,664321,5,219,NaN,NaN,31,41845,0,NaN,0,1.39,NaN,4.63,NaN,4.532599,0,14310,3,100,2,0,1,0,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True,NaN,4,1,21,0,18,5,122,3,NaN,-0.097401,NaN,False,25,25,1
9917528,664321,5,219,NaN,NaN,31,123467,3,5.0,0,2.20,NaN,4.97,NaN,4.465908,0,14310,3,100,2,0,1,0,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True,3.0,4,1,21,0,18,5,122,3,NaN,-0.504092,NaN,False,25,25,1


In [58]:
norm_srch_res_count_z_prop_id = make_aggregate_feature(df, ['prop_id'], 'srch_res_count', transforms=['med', 'std', 'z'], use_median=True)
norm_srch_res_count_z_prop_id

0.065 NAs


srch_res_count_med_prop_id  srch_res_count_std_prop_id   
srch_id prop_id                                                           
1       893                            34.0                    4.266150  \
        10404                          34.0                    4.521655   
        21315                          34.0                    4.498311   
        27348                          34.0                    3.500208   
        29604                          34.0                    4.624512   
...                                     ...                         ...   
665572  32019                          29.0                    5.835921   
        33959                          26.0                   10.327445   
        35240                          27.0                    8.259194   
        94437                          29.0                    7.127951   
        99509                          16.0                   10.983624   

                 srch_res_count_z_prop_id  
srch_id prop_id                            
1       893                     -1.406420  
        10404                   -1.326948  
        21315                   -1.333834  
        27348                   -1.714184  
        29604                   -1.297434  
...                                   ...  
665572  32019                   -3.769756  
        33959                   -1.839758  
        35240                   -2.421544  
        94437                   -3.086441  
        99509                   -0.819402  

[9917530 rows x 3 columns]

# Comments on aggregation over all ids (not just the training set)

When calculating averages over srch_destination_id or prop_id for instance, there is a significant number of them only encountered in the test set. Pedantically speaking we shouldn't use data from the test set in our model. However to get the best possible result in the competition we are going to leverage it.

If we would calculate means over ids from the training set only, then we would have NAs when predicting on the test set (there would be no NAs on the training set). At the very least, the validation set should have the same NA characteristics as the test set, but we can do better. We can remove aggregated values for part of the ids from the training set, thus the model learns not to rely on them too much (not more than what will be available in the test set) - make sure to discard only from the ids which don't appear in the test set or add the aggregated values back to the test set before prediction. This is expected to perform worse than using the statistics from the test set.

# Join all dataframes together

In [59]:
joined = df.set_index(['srch_id', 'prop_id'])
from tqdm import tqdm
for to_join in tqdm([
    norm_price_per_srch,
    tosub_prop_starrating_per_dest,# prop_starrating_w0_d_srch_destination_id
    norm_prop_starrating_per_srch, # prop_starrating_w0_d_srch_id

    tosub_prop_review_score_per_dest, # prop_review_score_d_srch_destination_id
    tosub_prop_review_score_per_dest_prop_starrating, # prop_review_score_d_srch_destination_id__prop_starrating
    norm_prop_review_score_per_srch, # prop_review_score_d_srch_id

    tosub_prop_location_score1_per_dest, # prop_location_score1_d_srch_destination_id
    tosub_prop_location_score1_per_dest_prop_starrating, # prop_location_score1_d_srch_destination_id__prop_starrating
    norm_prop_location_score1_per_srch, # prop_location_score1_d_srch_id

    tosub_prop_location_score2_per_dest_id, # prop_location_score2_d_srch_destination_id
    tosub_prop_location_score2_per_dest_id_prop_starrating, # prop_location_score2_d_srch_destination_id__prop_starrating
    norm_prop_location_score2_per_srch, # prop_location_score2_d_srch_id

    norm_price_per_search_prop_starrating,

    norm_prop_price_hist_advantage_per_srch_id,
    norm_visitor_price_hist_advantage_per_srch_id,
    norm_srch_query_affinity_score,
    norm_srch_res_count_z_prop_id
    ]):
    joined = joined.join(to_join, how='left')



100%|██████████| 17/17 [00:46<00:00,  2.73s/it]


In [60]:
# -1 to keep the sign
# we want to have the differences between the values relative to the search mean, and relative to the destination mean (someone might be looking at the 4 star hotels in an area predominantly with 3 star hotels, this might say something about the booking behaviour that cannot be seened simply by looking at the relative search orderings).

joined.prop_starrating_w0_d_srch_destination_id -= joined.prop_starrating_w0_d_srch_id * -1

joined.prop_review_score_d_srch_destination_id -= joined.prop_review_score_d_srch_id * -1
joined.prop_review_score_d_srch_destination_id__prop_starrating -= joined.prop_review_score_d_srch_id * -1

joined.prop_location_score1_d_srch_destination_id -= joined.prop_location_score1_d_srch_id * -1
joined.prop_location_score1_d_srch_destination_id__prop_starrating -= joined.prop_location_score1_d_srch_id * -1

joined.prop_location_score2_d_srch_destination_id -= joined.prop_location_score2_d_srch_id * -1
joined.prop_location_score2_d_srch_destination_id__prop_starrating -= joined.prop_location_score2_d_srch_id * -1

joined

,,site_id,visitor_location_country_id,visitor_hist_starrating,visitor_hist_adr_usd,prop_country_id,prop_starrating,prop_review_score,prop_brand_bool,prop_location_score1,prop_location_score2,prop_log_historical_price,position,price_usd,promotion_flag,srch_destination_id,srch_length_of_stay,srch_booking_window,srch_adults_count,srch_children_count,srch_room_count,srch_saturday_night_bool,srch_query_affinity_score,random_bool,comp1_rate,comp1_inv,comp1_rate_percent_diff,comp2_rate,comp2_inv,comp2_rate_percent_diff,comp3_rate,comp3_inv,comp3_rate_percent_diff,comp4_rate,comp4_inv,comp4_rate_percent_diff,comp5_rate,comp5_inv,comp5_rate_percent_diff,comp6_rate,comp6_inv,comp6_rate_percent_diff,comp7_rate,comp7_inv,comp7_rate_percent_diff,comp8_rate,comp8_inv,comp8_rate_percent_diff,click_bool,booking_bool,is_test,...,prop_count_per_srch_dest,srch_dest_count_per_prop,price_usd_std_srch_id,price_usd_med_srch_id,price_usd_d_srch_id,prop_starrating_w0_std_srch_destination_id,prop_starrating_w0_med_srch_destination_id,prop_starrating_w0_d_srch_destination_id,prop_starrating_w0_std_srch_id,prop_starrating_w0_med_srch_id,prop_starrating_w0_d_srch_id,prop_review_score_std_srch_destination_id,prop_review_score_med_srch_destination_id,prop_review_score_d_srch_destination_id,prop_review_score_std_srch_destination_id__prop_starrating,prop_review_score_med_srch_destination_id__prop_starrating,prop_review_score_d_srch_destination_id__prop_starrating,prop_review_score_std_srch_id,prop_review_score_med_srch_id,prop_review_score_d_srch_id,prop_location_score1_std_srch_destination_id,prop_location_score1_med_srch_destination_id,prop_location_score1_d_srch_destination_id,prop_location_score1_std_srch_destination_id__prop_starrating,prop_location_score1_med_srch_destination_id__prop_starrating,prop_location_score1_d_srch_destination_id__prop_starrating,prop_location_score1_std_srch_id,prop_location_score1_med_srch_id,prop_location_score1_d_srch_id,prop_location_score2_std_srch_destination_id,prop_location_score2_med_srch_destination_id,prop_location_score2_d_srch_destination_id,prop_location_score2_std_srch_destination_id__prop_starrating,prop_location_score2_med_srch_destination_id__prop_starrating,prop_location_score2_d_srch_destination_id__prop_starrating,prop_location_score2_std_srch_id,prop_location_score2_med_srch_id,prop_location_score2_d_srch_id,price_usd_std_srch_id__prop_starrating,price_usd_med_srch_id__prop_starrating,price_usd_d_srch_id__prop_starrating,price_hist_logdiff_std_srch_id,price_hist_logdiff_med_srch_id,price_hist_logdiff_d_srch_id,visitor_hist_adr_usd_logdiff_d_srch_id,srch_query_affinity_score_std_srch_id,srch_query_affinity_score_d_srch_id,srch_res_count_med_prop_id,srch_res_count_std_prop_id,srch_res_count_z_prop_id
srch_id,prop_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,893,12,187,NaN,NaN,219,3,3.5,1,2.83,0.0438,4.95,27.0,4.651767,0,23246,1,0,4,0,1,1,NaN,1,NaN,NaN,NaN,0.0,0.0,NaN,0.0,0.0,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN,0.0,0.0,False,...,27173,1237,0.390255,4.934823,-0.283055,0.808048,3.0,0.0,0.766356,3.0,0.0,0.527944,4.00,-0.75,0.461749,4.00,-0.75,0.533854,3.75,-0.25,0.667135,2.30,1.06,0.578988,2.200,1.160,0.518734,2.30,0.53,0.043762,0.0164,0.04115,0.030738,0.01640,0.04115,0.047217,0.03005,0.01375,0.252016,4.930148,-0.278381,0.415762,0.087924,-0.386156,NaN,NaN,NaN,34.0,4.26615,-1.406420
896,893,5,2,NaN,NaN,219,3,3.5,1,2.83,0.0438,4.98,24.0,4.836282,0,23246,1,16,1,0,1,1,NaN,0,NaN,NaN,NaN,0.0,0.0,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN,0.0,0.0,False,...,27173,1237,0.303866,4.686740,0.149542,0.808048,3.0,0.0,0.774597,3.0,0.0,0.527944,4.00,-1.00,0.461749,4.00,-1.00,0.536751,4.00,-0.50,0.667135,2.30,1.01,0.578988,2.200,1.110,0.621438,2.35,0.48,0.043762,0.0164,0.04100,0.030738,0.01640,0.04100,0.046093,0.03020,0.01360,0.205055,4.722469,0.113813,0.175557,-0.245794,0.102076,NaN,NaN,NaN,34.0,4.26615,0.46

# Save the new dataframe

In [61]:
joined.reset_index().to_csv('data/joined_all_features.csv.zip')

# use feather to save the joined dataframe to a binary file
#joined.reset_index().to_feather('data/joined_all_features.feather')

In [62]:
# cast all float in the joined dataframe to float16
joined = joined.astype({col: 'float16' for col in joined.columns if joined[col].dtype in ('float64', 'float32')})
import gc
gc.collect()

/home/robert/miniconda3/envs/dmt_expedia/lib/python3.11/site-packages/pandas/core/dtypes/astype.py:138: RuntimeWarning: overflow encountered in cast
  return arr.astype(dtype, copy=True)
/home/robert/miniconda3/envs/dmt_expedia/lib/python3.11/site-packages/pandas/core/dtypes/astype.py:138: RuntimeWarning: overflow encountered in cast
  return arr.astype(dtype, copy=True)
/home/robert/miniconda3/envs/dmt_expedia/lib/python3.11/site-packages/pandas/core/dtypes/astype.py:138: RuntimeWarning: overflow encountered in cast
  return arr.astype(dtype, copy=True)
/home/robert/miniconda3/envs/dmt_expedia/lib/python3.11/site-packages/pandas/core/dtypes/astype.py:138: RuntimeWarning: overflow encountered in cast
  return arr.astype(dtype, copy=True)
/home/robert/miniconda3/envs/dmt_expedia/lib/python3.11/site-packages/pandas/core/dtypes/astype.py:138: RuntimeWarning: overflow encountered in cast
  return arr.astype(dtype, copy=True)
/home/robert/miniconda3/envs/dmt_expedia/lib/python3.11/site-pack

74

In [ ]:
joined.reset_index().to_csv('data/joined_all_features_fp16.csv.zip')